# Filtering variants

This is a [Jupyter notebook](https://jupyter.org/) containing a guide for filtering variant call (vcf) files that have been annotated using [SNPEff](https://pcingola.github.io/SnpEff/).

This workflow was developed on output from the nfcore/sarek pipeline (https://nf-co.re/sarek) which is a variant calling and annotation pipeline built in the [Nextflow](https://www.nextflow.io/) workflow manager system. Sarek includes SNPEff annotation as part of the workflow. However, any SNPEff annotated vcf file can be used as input for this workflow, not just those generated by nfcore/sarek.

This workflow was prepared by the [eResearch Office, QUT.](https://qutvirtual4.qut.edu.au/group/staff/governance/organisational-structure/academic-division/research-portfolio/research-infrastructure/eresearch)

**********************************

# Contents
[1. How to use this notebook](#overview)

[2. Requirements](#require)

[3. Installing SNPEff](#install)

[4. Filtration principles](#filt)

[5. Filter by impact](#impact)

[6. Look for overlaps](#overlap)

[7. Merging vcfs](#merge)

[7. Output as table](#table)


***************************

## How to use this notebook <a class="anchor" id="overview"></a>

Juypter Notebooks run a 'kernel' that allow code to be run in code 'cells' in the Notebook. This Notebook is running the BASH kernel, which allows for commands to be run on QUTs HPC.

You can run a code cell by clicking on the cell itself and clicking the run button (at the top of this Notebook), or by pressing shift+enter.

<div class="alert alert-block alert-warning">
As an example, run the following code cell to list the contents of your HPC home directory.
</div>

In [ ]:
ls $HOME

Before every code cell is a colour-coded text box that tells you what the code cell will do. 

<div class="alert alert-block alert-success">
A green text box indicates a code cell that must be run, without alteration, to complete the workflow.
</div>

<div class="alert alert-block alert-warning">
A yellow text box indicates an optional code cell that doesn't have to be run to complete the workflow, but can be run to complete optional tasks.
</div>

<div class="alert alert-block alert-info">
A blue text box indicates a code cell that requires user input - it must be run to complete the workflow and the user needs to modify the command in the cell. This is required for user-specific or project-specific operations, such as moving to a directory that contains the project data files.
</div>

In this guide you will find instructions and code cells to run that will filter vcf files that were annoated by SNPeff.

*******************************

## Requirements <a class="anchor" id="require"></a>

You will need a QUT HPC account. If you are seeing this Notebook, you most likely already have a HPC account. Regardless, you can request an account be created, or request any other HPC or bioinformatics support, via the portal here: https://eresearchqut.atlassian.net/servicedesk/customer/portals

You will also need to have all the annotated vcf files in a directory on the HPC. Sarek outputs these files as '..samplename..'snpEff.ann.vcf.gz. Depending on how you ran sarek or how your data is organised, these may be in separate directories based on treatment group or individual samples. 

It is recommended to copy all these files to a single directory. As directory and file structure varies per project, it's not possible to include the code to achieve this in this Notebook and thus the user must do this manually prior to running this workflow.

A guide to copying and moving data via the Linux command line is [here](https://www.thegeekdiary.com/linux-command-line-basics-for-beginners-copy-and-move-files-and-directories-cp-and-mv-commands/), or you can contact the HPC staff to locate/move your data files via the portal [here](https://eresearchqut.atlassian.net/servicedesk/customer/portals)

<div class="alert alert-block alert-info">
Enter the directory that contains your SNPEff annotated vcf files. You will need to change the directory path to where you moved your '..samplename..'snpEff.ann.vcf.gz files to. You can find this path by typing 'pwd' on the command-line when you are in that directory, or by contacting the HPC staff via the portal (above link). The structure of the below command should be `root_path=/directory/containing/my/vcf/files`.
</div>

In [ ]:
root_path=/work/liver/nextflow/sarek/individual/sarek_VCFs_annotation/All_samples

<div class="alert alert-block alert-success">
Now move to the above directory (cd = change directory): 
</div>

In [ ]:
cd $root_path

<div class="alert alert-block alert-warning">
To see if you are in the correct directory, run the 'ls' code cell below. You should see a list of all your SNPEff annotated vcf files. If you don't see the files, you've entered the above location incorrectly and need to correct and re-run the above code cell.
</div>

In [ ]:
ls

****************************

## Installing SNPEff <a class="anchor" id="install"></a>

SNPEff needs to be installed first to run the various SNPEff command-line filtration options.

<div class="alert alert-block alert-success">
Create a new directory called 'snpeff' and download the latest version of SNPEff to that directory, then unzip the file.
</div>

In [ ]:
mkdir snpeff
cd snpeff
wget https://snpeff.blob.core.windows.net/versions/snpEff_latest_core.zip
unzip -q snpEff_latest_core.zip
cd $root_path

<div class="alert alert-block alert-success">
SNPEff is based on Java, so the Java module will need to be loaded before running SNPEff.
</div>

In [ ]:
module load java

<div class="alert alert-block alert-success">
Check the version of SNPEff, to make sure it has been installed correctly
</div>

In [ ]:
java -jar ./snpeff/snpEff/snpEff.jar -version

********************

## Filtration principles <a class="anchor" id="filt"></a>

<div class="alert alert-block alert-warning">
If you look in the installed SNPEff directory (you can run the code cell below to do this), you'll see the two main tools: snpEff.jar and SnpSift.jar.
    </div>

In [ ]:
ls ./snpeff/snpEff/

snpEff.jar
> is a variant annotation and effect prediction tool. It annotates and predicts the effects of genetic variants (such as amino acid change)

SnpSift.jar
>is a toolbox that allows you to filter and manipulate annotated files.

snpEff has already been run on your samples, as part of the nfcore/sarek workflow, creating the '..samplename..'snpEff.ann.vcf.gz files we're working with here. 

In this Notebook we will be using SnpSift.jar to filter your samples by certain parameters, such as high impact variants.

SnpSift.jar is highly flexible and can be used to filter by quality, read depth, genotype and any of the annotation fields generated by snpEff.jar (such as frameshift mutations, synonymous, missense, splice regions, intron, exon, etc, etc).

A list and explanation of the filtration options can be seen here:

https://pcingola.github.io/SnpEff/ss_filter/

And a list of the possible annotation fields can be seen here:

https://pcingola.github.io/SnpEff/adds/VCFannotationformat_v1.0.pdf

<div class="alert alert-block alert-warning">
By following this guide, you have installed SnpSift.jar in ./snpeff/snpEff/SnpSift.jar and can run any of the examples in the link above by adding './snpeff/snpEff/' before 'SnpSift.jar' in any command.

For example, if you wanted to run the first example in https://pcingola.github.io/SnpEff/ss_filter/ (filter out samples with quality less than 30), you could run the following (change 'variants.vcf' to one of your variant files and 'filtered.vcf' to a suitable name for the output file):
</div>

In [ ]:
cat variants.vcf | java -jar ./snpeff/snpEff/SnpSift.jar filter " ( QUAL >= 30 )" > filtered.vcf

SnpSift.jar can do more than filter samples, it can extract specific fields from your annotated vcf file, join by genomic regions, split by chomosome, etc. See all the SnpSift.jar commands here:  https://pcingola.github.io/SnpEff/ss_introduction/

***************************

## Filter by impact <a class="anchor" id="impact"></a>

SNPEff annotates variants by HIGH, MODERATE, LOW or MODIFIER impact.

https://pcingola.github.io/SnpEff/se_inputoutput/#impact-prediction

>HIGH	The variant is assumed to have high (disruptive) impact in the protein, probably causing protein truncation, loss of function or triggering nonsense mediated decay.

>MODERATE	A non-disruptive variant that might change protein effectiveness.

>LOW	Assumed to be mostly harmless or unlikely to change protein behavior.

>MODIFIER	Usually non-coding variants or variants affecting non-coding genes, where predictions are difficult or there is no evidence of impact.

A list of all the effects (e.g. deletions, duplications, frame shift, etc) can be seen here, including if their impact category: https://pcingola.github.io/SnpEff/se_inputoutput/#effect-prediction-details

<div class="alert alert-block alert-success">
Create a main directory to store filtered results from each sample group, which will be in their own sample group subdirectories:
</div>

In [ ]:
cd $root_path
mkdir Filtered

You can filter out variants by impact (e.g. only keep high impact variants).

<div class="alert alert-block alert-success">
Run one or more of the three following code cells. The first cell filters out HIGH impact variants, the second cell MODERATE and the third cell LOW. You may only be interested in HIGH impact variants, in which case just run the first cell. Note: this will not alter the original vcf files. New vcf files will be created in 'filtered_high', 'filtered_moderate' or 'filtered_low' directories. These commands may take several minutes to run.
</div>

In [ ]:
mkdir Filtered/filtered_high
for f1 in *.vcf*
do
echo filtering $f1
java -jar ./snpeff/snpEff/SnpSift.jar filter "ANN[*].IMPACT has 'HIGH'" $f1 > "Filtered/filtered_high/high_$f1"
done

In [ ]:
mkdir Filtered/filtered_moderate
for f1 in *.vcf*
do
echo filtering $f1
java -jar ./snpeff/snpEff/SnpSift.jar filter "ANN[*].IMPACT has 'MODERATE'" $f1 > "Filtered/filtered_moderate/moderate_$f1"
done

In [ ]:
mkdir Filtered/filtered_low
for f1 in *.vcf*
do
echo filtering $f1
java -jar ./snpeff/snpEff/SnpSift.jar filter "ANN[*].IMPACT has 'LOW'" $f1 > "Filtered/filtered_low/low_$f1"
done

<div class="alert alert-block alert-warning">
To check that the filtered vcf files were created by listing the files in the 'filtered_high' directory. You can check the MODERATE of LOW results also by changing 'filtered_high' to 'filtered_moderate' or 'filtered_low' in the code cell below.
</div>

In [ ]:
ls Filtered/filtered_high

*******************

## Look for overlaps <a class="anchor" id="overlap"></a>

Usually you will have multiple samples per treatment group, which means you'll want to find common variants across multiple samples. This is done by creating intersections between two or more vcf files.

<div class="alert alert-block alert-success">
We'll be using bcftools to look for intersects. bcftools is already installed on the HPC as a module. Load the bcftools module:
</div>

In [ ]:
module load bcftools

<div class="alert alert-block alert-success">
Create a main directory to store each sample group intersect results, which will be in their own sample group subdirectories:
</div>

In [ ]:
cd $root_path
mkdir Intersect

<div class="alert alert-block alert-info">
Now select a target treatment group.
In the code cell below, enter a character string that is unique to the target treatment group, i.e. is used in the name of every sample of only that group (i.e. change 'group=...' to the target group name your sample names).
</div>

In [ ]:
group=ALD_EtOH

<div class="alert alert-block alert-warning">
If you can't remember your sample names, run 'ls' to see them:
</div>

In [ ]:
ls

<div class="alert alert-block alert-warning">
You can also check that you are only pulling out the correct samples. The samples you will be intersecting are: 
</div>

In [ ]:
ls *$group*

<div class="alert alert-block alert-success">
Bcftools requires that the vcf file is compressed and indexed. First we'll compress the vcf files:
</div>

In [ ]:
mkdir Intersect/$group"_intersect"
for f1 in *$group*
do
echo compressing $f1
bgzip -c $f1 > Intersect/$group"_intersect"/$f1".gz"
done

<div class="alert alert-block alert-success">
Then we'll index the vcf files:
</div>

In [ ]:
cd Intersect/$group"_intersect"
for f1 in *$group*".gz"
do
echo indexing $f1
bcftools index $f1
done
cd $root_path

<div class="alert alert-block alert-info">
Now we can check for intersections between these samples:
</div>

In [ ]:
cd Intersect/$group"_intersect"
bcftools isec -n+7 -c all *$group*".gz" > $group"_sharedPositions.txt"
cd $root_path

**Note: the '-n+7' option says to output variants that intersect in 7 or more of your samples. You should adjust this to a number suitable to the total number of samples in the treatment group. For example, if you have n=20, setting -n+10 would output variants that are in 50% of your samples. -n+20 would only output variants that are in all samples, -n+2 in 2 or more samples, etc.**

<div class="alert alert-block alert-warning">
Find the number of samples in your treatment group:
</div>

In [ ]:
ls Intersect/$group"_intersect"/*$group*".gz" | wc -l

Now that you have a table of intersecting variants, you need to pull out just these intersects from your vcf files. For this we use bcftools view:

In [ ]:
cd $root_path
cd Intersect/$group"_intersect"
for f1 in *$group*".gz"
do
echo filtering $f1
bcftools view -T $group"_sharedPositions.txt" $f1 -Oz > "sharedPositions_"$f1
done
cd $root_path

These filtered vcfs need to be re-indexed:

In [ ]:
cd Intersect/$group"_intersect"
for f1 in sharedPositions*.vcf.gz
do
echo indexing $f1
bcftools index $f1
done
cd $root_path

You can find intersects in other groups by going to the treatment group selection code ('group=..'), changing the group name and then running through the process again. You can do this for each treatment group. 

*******************

## Merging vcfs <a class="anchor" id="merge"></a>

Once you have filtered and/or found intersecting variants between samples, you can merge all treatment group samples into a single vcf file. We'll be using bcftools to accomplish this. If you've started a new session, make sure you run the 'module load bcftools' code cell in the previous section.

<div class="alert alert-block alert-success">
Create a main directory to store merged vcf files for each sample group, which will be in their own sample group subdirectories:
</div>

In [ ]:
cd $root_path
mkdir Merged

<div class="alert alert-block alert-info">
Enter a subdirectory that contains your intersected results
</div>

In [ ]:
intdir=ALD_EtOH_intersect

<div class="alert alert-block alert-warning">
See here for a list of the intersected samples group directories:
</div>

In [ ]:
cd $root_path
ls -d Intersect/*/ | cut -f2 -d'/'

<div class="alert alert-block alert-success">
Use bcftools merge to merge the vcf files:
</div>

In [ ]:
bcftools merge Intersect/$intdir/sharedPositions*.vcf.gz > Merged/$intdir"merged.vcf"

*************

## Output as table <a class="anchor" id="table"></a>

SnpSift.jar can extract any fields in a vcf, including any annotation fields, outputting this as a table. This is useful for importing your filtered results into another program like R or Excel.

https://pcingola.github.io/SnpEff/ss_extractfields/

<div class="alert alert-block alert-warning">
For example, if you wanted to output chromosome, position, ID and allele frequency from a vcf file, you could run the following (change 'variants.vcf' to one of your vcf files, and 'table.txt' to a more informative name):
</div>

In [ ]:
java -jar ./snpeff/snpEff/SnpSift.jar extractFields variants.vcf CHROM POS ID AF > table.txt

<div class="alert alert-block alert-warning">
Then you can either download the table or view it like so (change 'table.txt' to the same output filename that you used above)
</div>

In [ ]:
cat table.txt